In [1]:
import tensorflow as tf
from tensorflow.keras import datasets, layers, models, optimizers
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator

CIFAR-10数据集包含10个类别，每个类别有6000张图片，每张照片大小为32*32，颜色为RGB。

In [2]:
IMG_CHANNELS = 3
IMG_ROWS = 32
IMG_COLS = 32

BATCH_SIZE = 128
EPOCHS = 30
CLASSES = 10
VALIDATION_SPLIT = 0.2
OPTIM = tf.keras.optimizers.Adam()

In [3]:
(x_train, y_train), (x_test, y_test) = datasets.cifar10.load_data()
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
# normalize
mean = np.mean(x_train, axis=(0, 1, 2, 3))
std = np.std(x_train, axis=(0, 1, 2, 3))
y_train = tf.keras.utils.to_categorical(y_train, CLASSES)
y_test = tf.keras.utils.to_categorical(y_test, CLASSES)

In [4]:
def build(input_shape, classes):
    model = models.Sequential()
    # 卷积层
    model.add(layers.Convolution2D(32, (3, 3), activation='relu', input_shape=input_shape))
    # 池化层
    model.add(layers.MaxPooling2D(pool_size=(2, 2)))
    # 失活层
    model.add(layers.Dropout(0.25))
    # 稠密网络
    model.add(layers.Flatten())
    model.add(layers.Dense(512, activation='relu'))
    model.add(layers.Dropout(0.5))
    model.add(layers.Dense(classes, activation='softmax'))
    return model

In [5]:
callbacks = [tf.keras.callbacks.TensorBoard(log_dir='./CIFAR-10/logs', histogram_freq=1)]

In [6]:
model = build(input_shape=(IMG_ROWS, IMG_COLS, IMG_CHANNELS), classes=CLASSES)
model.compile(loss='categorical_crossentropy', optimizer=OPTIM, metrics=['accuracy'])
model.fit(x_train, y_train, 
          batch_size=BATCH_SIZE,
          epochs=EPOCHS, 
          validation_split=VALIDATION_SPLIT, 
          callbacks=callbacks)

Epoch 1/30
313/313 [==============================] - 22s 67ms/step - loss: 10.6620 - accuracy: 0.1145 - val_loss: 2.1766 - val_accuracy: 0.1649
Epoch 2/30
313/313 [==============================] - 20s 64ms/step - loss: 2.1981 - accuracy: 0.1553 - val_loss: 1.9980 - val_accuracy: 0.2292
Epoch 3/30
313/313 [==============================] - 22s 70ms/step - loss: 2.0691 - accuracy: 0.2079 - val_loss: 1.9723 - val_accuracy: 0.2651
Epoch 4/30
313/313 [==============================] - 22s 72ms/step - loss: 1.9825 - accuracy: 0.2416 - val_loss: 1.7692 - val_accuracy: 0.3382
Epoch 5/30
313/313 [==============================] - 21s 68ms/step - loss: 1.9119 - accuracy: 0.2686 - val_loss: 1.7226 - val_accuracy: 0.3514
Epoch 6/30
313/313 [==============================] - 20s 64ms/step - loss: 1.8594 - accuracy: 0.2946 - val_loss: 1.6892 - val_accuracy: 0.3752
Epoch 7/30
313/313 [==============================] - 20s 64ms/step - loss: 1.8038 - accuracy: 0.3178 - val_loss: 1.6106 - val_accuracy

In [7]:
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Test loss: 1.2007423639297485
Test accuracy: 0.5921000242233276


使用更深的网络提高CIFAE-10的性能

In [8]:
def build_more():
    model = models.Sequential()
    model.add(layers.Convolution2D(32, (3, 3), padding='same', activation='relu', input_shape=(IMG_ROWS, IMG_COLS, IMG_CHANNELS)))
    model.add(layers.BatchNormalization())
    model.add(layers.Convolution2D(32, (3, 3), padding='same', activation='relu'))
    model.add(layers.BatchNormalization())
    model.add(layers.MaxPooling2D(pool_size=(2, 2)))
    model.add(layers.Dropout(0.2))
    
    model.add(layers.Convolution2D(64, (3, 3), padding='same', activation='relu'))
    model.add(layers.BatchNormalization())
    model.add(layers.Convolution2D(64, (3, 3), padding='same', activation='relu'))
    model.add(layers.BatchNormalization())
    model.add(layers.MaxPooling2D(pool_size=(2, 2)))
    model.add(layers.Dropout(0.3))
    
    model.add(layers.Convolution2D(128, (3, 3), padding='same', activation='relu'))
    model.add(layers.BatchNormalization())
    model.add(layers.Convolution2D(128, (3, 3), padding='same', activation='relu'))
    model.add(layers.BatchNormalization())
    model.add(layers.MaxPooling2D(pool_size=(2, 2)))
    model.add(layers.Dropout(0.4))
    
    model.add(layers.Flatten())
    model.add(layers.Dense(10, activation='softmax'))
    return model

In [9]:
model_more = build_more()
model_more.compile(loss='categorical_crossentropy', optimizer='RMSprop', metrics=['accuracy'])
batch_size = 64
model_more.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_1 (Conv2D)           (None, 32, 32, 32)        896       
                                                                 
 batch_normalization (BatchN  (None, 32, 32, 32)       128       
 ormalization)                                                   
                                                                 
 conv2d_2 (Conv2D)           (None, 32, 32, 32)        9248      
                                                                 
 batch_normalization_1 (Batc  (None, 32, 32, 32)       128       
 hNormalization)                                                 
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 16, 16, 32)       0         
 2D)                                                             
                                                      

In [10]:
EPOCHS = 50
model_more.fit(x_train, y_train,
               batch_size=batch_size,
               epochs=EPOCHS,
               validation_data=(x_test, y_test),)

Epoch 1/50
782/782 [==============================] - 154s 195ms/step - loss: 1.7693 - accuracy: 0.4867 - val_loss: 1.7343 - val_accuracy: 0.5084
Epoch 2/50
782/782 [==============================] - 154s 197ms/step - loss: 1.1782 - accuracy: 0.6471 - val_loss: 1.0065 - val_accuracy: 0.6694
Epoch 3/50
782/782 [==============================] - 156s 199ms/step - loss: 0.9626 - accuracy: 0.7023 - val_loss: 1.0851 - val_accuracy: 0.6683
Epoch 4/50
782/782 [==============================] - 156s 200ms/step - loss: 0.8076 - accuracy: 0.7357 - val_loss: 0.8692 - val_accuracy: 0.7205
Epoch 5/50
782/782 [==============================] - 135s 173ms/step - loss: 0.7046 - accuracy: 0.7616 - val_loss: 0.7010 - val_accuracy: 0.7640
Epoch 6/50
782/782 [==============================] - 149s 190ms/step - loss: 0.6313 - accuracy: 0.7861 - val_loss: 0.6762 - val_accuracy: 0.7676
Epoch 7/50
782/782 [==============================] - 154s 198ms/step - loss: 0.5770 - accuracy: 0.8029 - val_loss: 0.8764 -

In [11]:
score = model_more.evaluate(x_test, y_test, batch_size=batch_size)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

157/157 [==============================] - 6s 36ms/step - loss: 0.5932 - accuracy: 0.8423
Test loss: 0.5932190418243408
Test accuracy: 0.8422999978065491
